In [7]:
import cv2
import time 
import ipywidgets as widgets 
import IPython.display as display 
import copy
from ipywidgets import Video, Image
from IPython.display import display 
from PIL import Image 
import numpy as np
import cv2
import base64
import ipywidgets as widgets
import copy
import time
import mediapipe as mp
import IPython

In [8]:
url = "http://114.70.235.37:8091/?action=stream"

In [9]:
import cv2
import mediapipe as mp
import numpy as np
from matplotlib import pyplot as plt

In [10]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

BG_COLOR = (192, 192, 192) # gray

In [11]:
def pose(image):

    with mp_pose.Pose(
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5) as pose:

        # 필요에 따라 성능 향상을 위해 이미지 작성을 불가능함으로 기본 설정합니다.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(image)

        # 포즈 주석을 이미지 위에 그립니다.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        mp_drawing.draw_landmarks(
            image,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())

        """
        if results.pose_landmarks != None:
            print(type(results.pose_landmarks))
            break
        """

        # 보기 편하게 이미지를 좌우 반전합니다.
        return image

Image(value=b'', layout="Layout(border='solid')")

NameError: name 'cap' is not defined

In [12]:
import torch

# Model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')
model.conf = 0.6

# Images
dir = 'https://github.com/ultralytics/yolov5/raw/master/data/images/'
imgs = [dir + f for f in ('zidane.jpg', 'bus.jpg')]  # batch of images

# Inference
results = model(imgs)
results.print()  # or .show(), .save()

Using cache found in /home/stu1/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2022-5-22 Python-3.8.12 torch-1.11.0 CUDA:0 (NVIDIA GeForce RTX 3080 Ti, 12045MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


image 1/2: 720x1280 2 persons, 1 tie
image 2/2: 1080x810 3 persons, 1 bus
Speed: 615.1ms pre-process, 2.9ms inference, 0.6ms NMS per image at shape (2, 3, 640, 640)


In [13]:
results = model(img)
results.print() 

NameError: name 'img' is not defined

In [ ]:
cap = cv2.VideoCapture(url)

wImg = widgets.Image( 
    layout = widgets.Layout(border="solid") 
) 
display(wImg)

if cap.isOpened(): 
    ret, img = cap.read()
    while ret:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        # 동영상 파일에서 캡쳐된 이미지를 이미지 파일 스트림으로 다시 인코딩을 한다. 
        results = model(img)
        
        # xy에는 인식한 각 객체, xy 좌표로 구성된 데이터 프레임 담김
        xy = results.pandas().xyxy[0]
        
        # person에는 인식한 객체중 'person'만 추출
        person = xy[xy['name'] == 'person']
        
        crop_image = []
        
        # 이미지 
        for index, row in person.iterrows():
            # 이미지에서 사람에 해당하는 부분만 크롭
            # 사진 한장(img) -> 사람 사진 여러장(crop_image[])
            crop_image.append(img[int(row['ymin']):int(row['ymax']), int(row['xmin']):int(row['xmax'])])

        
        for croped in crop_image:
            croped = pose(croped)
            
            # 포즈 결과 출력
            tmpStream = cv2.imencode(".jpeg", croped)[1].tobytes()
            wImg.value = tmpStream 

        
        ret, img = cap.read()
else:
    print("not opened")
cap.release()